In [2]:
import pandas as pd
import numpy as np
from IPython.display import display

import matplotlib as plt
import seaborn as sns

%matplotlib inline

In [3]:
# import scraped search data to dataframe
search = pd.read_csv('Data/bozeman_search_housing.csv', delimiter=',')

In [4]:
# create df of listing urls into csv
urls = search['detailUrl']
urls.to_csv('data/bozeman_housing_urls.csv', index=False)

In [5]:
# import scraped listing data (from individual URL) to dataframe
df = pd.read_csv('Data/bozeman_housing_info.csv', delimiter=',', low_memory=False)

In [6]:
# view columns of listing data
df.info()
print(df.columns.tolist())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Columns: 2000 entries, abbreviatedAddress to zpid
dtypes: bool(108), float64(757), int64(43), object(1092)
memory usage: 10.7+ MB
['abbreviatedAddress', 'adTargets/aamgnrc1', 'adTargets/bd', 'adTargets/city', 'adTargets/cnty', 'adTargets/dma', 'adTargets/fsbid', 'adTargets/guid', 'adTargets/listtp', 'adTargets/lot', 'adTargets/mlat', 'adTargets/mlong', 'adTargets/pid', 'adTargets/prange', 'adTargets/premieragent', 'adTargets/price', 'adTargets/proptp', 'adTargets/sqft', 'adTargets/sqftrange', 'adTargets/ssid', 'adTargets/state', 'adTargets/tflag', 'adTargets/yrblt', 'adTargets/zip', 'adTargets/zusr', 'address/city', 'address/community', 'address/neighborhood', 'address/state', 'address/streetAddress', 'address/subdivision', 'address/zipcode', 'apartmentsForRentInZipcodeSearchUrl/path', 'attributionInfo/agentEmail', 'attributionInfo/agentLicenseNumber', 'attributionInfo/agentName', 'attributionInfo/agentPhoneNumber'

In [7]:
# create important feature keyword list
relevant_keywords = [
    'streetAddress', 'city', 'state', 'zipcode', 'neighborhood',
    'latitude', 'longitude',
    'price', 'currency',
    'bedrooms', 'bathrooms', 'livingArea', 'lotSize', 'garage', 'cooling', 'heating', 'hasPrivatePool', 'water', 'levels',
    'yearBuilt',
    'homeType', 'propertyTypeDimension',
    'homeStatus', 'listingTypeDimension',
    'is_FSBO', 'is_newHome', 'is_foreclosure', 'is_FSBA',
    'isPremierBuilder', 'isShowcaseListing'
]

# Match columns that end with or contain one of the keywords
relevant_columns = [col for col in df.columns if any(kw in col for kw in relevant_keywords)]

# Extract just these columns
df_model = df[relevant_columns].copy()

# Flatten column names by taking just the last part (after the last '/')
df_model.columns = [col.split('/')[-1] for col in df_model.columns]

# Drop exact duplicates (e.g., multiple "price" columns from different propertyDetails indices)
df_model = df_model.loc[:, ~df_model.columns.duplicated()]

# Drop rows with missing target values (price)
df_model = df_model.dropna(subset=['price'])


In [8]:
# view filtered data set from keywords
print(df_model.head())
print(df_model.columns.tolist())

      city    price state  neighborhood        streetAddress  zipcode  \
0  Bozeman  1250000    MT           NaN  8373 Goldenstein Ln    59715   
1  Bozeman  1389000    MT           NaN    1440 Ryun Sun Way    59718   
2  Bozeman  1925000    MT           NaN   4054 Rain Roper Dr    59715   
3  Bozeman   559000    MT           NaN   2603 Snapdragon St    59718   
4  Bozeman  3850000    MT           NaN        3640 Jagar Ln    59718   

   bathrooms  bedrooms  cityId          path  ... livingAreaRangeUnits  \
0        4.0       5.0   44281  /bozeman-mt/  ...          Square Feet   
1        3.0       5.0   44281  /bozeman-mt/  ...          Square Feet   
2        4.0       3.0   44281  /bozeman-mt/  ...          Square Feet   
3        2.0       4.0   44281  /bozeman-mt/  ...          Square Feet   
4        3.0       4.0   44281  /bozeman-mt/  ...          Square Feet   

  lotSizeDimensions openParkingCapacity parkingCapacity waterBodyName  \
0               NaN                 NaN    

In [9]:
# select important columns into final data set
df_model = df_model[['levels','price', 'streetAddress', 'city', 'state', 'zipcode','latitude','longitude', 'bedrooms', 'bathrooms', 'homeStatus', 'homeType','livingArea','lotSize','waterView','hasPrivatePool','garageParkingCapacity','yearBuilt','is_FSBA', 'is_FSBO', 'is_foreclosure', 'is_newHome']]
print(df_model.head(100))

         levels    price            streetAddress     city state  zipcode  \
0           Two  1250000      8373 Goldenstein Ln  Bozeman    MT    59715   
1           Two  1389000        1440 Ryun Sun Way  Bozeman    MT    59718   
2           Two  1925000       4054 Rain Roper Dr  Bozeman    MT    59715   
3           Two   559000       2603 Snapdragon St  Bozeman    MT    59718   
4           Two  3850000            3640 Jagar Ln  Bozeman    MT    59718   
..          ...      ...                      ...      ...   ...      ...   
95          Two  1450000  605 E Cameron Bridge Rd  Bozeman    MT    59718   
96          Two   629000            4271 Cover St  Bozeman    MT    59718   
97  Multi/Split   834000          515 W Arnold St  Bozeman    MT    59715   
98          NaN    49995           164 Gareth Way  Bozeman    MT    59718   
99          One   885000     1037 Boylan Rd APT 1  Bozeman    MT    59715   

     latitude   longitude  bedrooms  bathrooms  ... livingArea   lotSize  \

In [10]:
# Clean Levels

print(df_model['levels'].unique())

df_model['levels_clean'] = (
    df_model['levels'].astype(str).str.strip().str.lower().replace({
        '1.0': 'One', 'one': 'One',
        '2.0': 'Two', 'two': 'Two',
        'three or more': 'Three Or More',
        'multi/split': 'Split'
    })
)

# Optional: Handle unexpected values as 'Unknown'
df_model['levels_clean'] = df_model['levels_clean'].where(
    df_model['levels_clean'].isin(['One', 'Two', 'Three Or More', 'Split']),
    'Unknown'
)

# Step 2: One-hot encode cleaned levels
levels_dummies = pd.get_dummies(df_model['levels_clean'], prefix='level')

# Step 3: Add one-hot columns to DataFrame and drop old 'levels' column
df_model = pd.concat([df_model.drop(columns=['levels']), levels_dummies], axis=1)

df_model = df_model.drop(columns = 'levels_clean')

# view filtered data set from keywords
print(df_model.head(100))
print(df_model.columns.tolist())

['Two' nan 'One' 'Multi/Split' 'Three Or More' 'Other' '2.0' '1.0']
      price            streetAddress     city state  zipcode   latitude  \
0   1250000      8373 Goldenstein Ln  Bozeman    MT    59715  45.639770   
1   1389000        1440 Ryun Sun Way  Bozeman    MT    59718  45.697390   
2   1925000       4054 Rain Roper Dr  Bozeman    MT    59715  45.678207   
3    559000       2603 Snapdragon St  Bozeman    MT    59718  45.676370   
4   3850000            3640 Jagar Ln  Bozeman    MT    59718  45.667190   
..      ...                      ...      ...   ...      ...        ...   
95  1450000  605 E Cameron Bridge Rd  Bozeman    MT    59718  45.713547   
96   629000            4271 Cover St  Bozeman    MT    59718  45.676370   
97   834000          515 W Arnold St  Bozeman    MT    59715  45.651897   
98    49995           164 Gareth Way  Bozeman    MT    59718        NaN   
99   885000     1037 Boylan Rd APT 1  Bozeman    MT    59715  45.681232   

     longitude  bedrooms  bathr